In [1]:
import numpy as np 
def get_Q_sls_model(Q):

    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref * 0
    NSLS = len(y)

    for i in range(NSLS):
        y[i] = y_sls_ref[i] / Q 
    dy[0] = 1. + 0.5 * y[0]

    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]

    #// copy to y_sls/w_sls
    w_sls = w_sls_ref * 1. 
    y_sls = dy * y 

    return w_sls,y_sls 


def compute_q_sls_model(y_sls,w_sls,om,exact=False):
    Q_ls = 1. 
    nsls = len(y_sls)
    if exact:
        for p in range(nsls):
            Q_ls += y_sls[p] * om**2 / (om**2 + w_sls[p]**2)

    # denom
    Q_demon = 0.
    for p in range(nsls):
        Q_demon += y_sls[p] * om * w_sls[p] / (om**2 + w_sls[p]**2)
    
    return Q_ls / Q_demon

def get_sls_modulus_factor(freq,Q):
    om = 2 * np.pi * freq

    w_sls,y_sls = get_Q_sls_model(Q)
    s = np.sum(1j * om * y_sls / (w_sls + 1j * om))

    return s + 1.

def get_sls_Q_deriv(freq,Q):
    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref  / Q 

    # corrector
    NSLS = len(y_sls_ref)
    dy[0] = 1. + 0.5 * y[0]
    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]
    dd_dqi = dy * 0
    dd_dqi[0] = 0.5 * y_sls_ref[0]
    for i in range(1,NSLS):
        dd_dqi[i] = dd_dqi[i-1] + (dy[i-1] - 0.5) * y_sls_ref[i-1] + dd_dqi[i-1] * y[i-1] +  0.5 * y_sls_ref[i]
        
    
    om = 2 * np.pi * freq
    #dd_dqi = dd_dqi * y + dy * y_sls_ref
    #dsdqi = np.sum(1j * om * dd_dqi /(w_sls_ref + 1j * om))
    dsdqi = 0. 
    for i in range(NSLS):
        dyp_dqi = dd_dqi[i] * y[i] + dy[i] * y_sls_ref[i]
        dsdqi += 1j * om * dyp_dqi / (w_sls_ref[i] + 1j * om);
    s = np.sum(1j * om * y * dy / (w_sls_ref + 1j * om))

    return s + 1., dsdqi

In [6]:
def love_func(c,r1,r2,L1_r,L2_r,N1_r,N2_r,Qv1,Qv2,Qh1,Qh2,H,om):
    freq = om / np.pi / 2

    # factor and deriv
    sn1,dsdqin1 = get_sls_Q_deriv(freq,Qh1)
    sn2,dsdqin2 = get_sls_Q_deriv(freq,Qh2)
    sl1,dsdqil1 = get_sls_Q_deriv(freq,Qv1)
    sl2,dsdqil2 = get_sls_Q_deriv(freq,Qv2)

    # complex modulus
    L1 = L1_r * sl1
    L2 = L2_r * sl2
    N1 = N1_r * sn1
    N2 = N2_r * sn2

    bv1 = np.sqrt(L1 / r1)
    bv2 = np.sqrt(L2 / r2)
    bh1 = np.sqrt(N1 / r1)
    bh2 = np.sqrt(N2 / r2)

    k = om / c
    t1 = np.sqrt(c**2 - bh1**2)
    t2 = np.sqrt(bh2**2 - c**2) 
    temp = np.tan(k * H / bv1 * t1)
    f = L2 / (L1) * t2 / bv2
    f -= t1 / bv1 * temp 
    f = L2/L1 * np.sqrt((N2/L2) - (r2 * c**2)/L2) - np.sqrt((c**2 *r1)/L1 - (N1/L1)) * np.tan((om * H)/c *np.sqrt((c**2 * r1)/L1 - (N1/L1)))

    # d(\tilde{c})/dL1/L2/N1/N2/rho
    dcc_dcL2=(-L2*(-1/2*N2/L2**2 + (1/2)*c**2*r2/L2**2)/(L1*np.sqrt(N2/L2 - c**2*r2/L2)) - np.sqrt(N2/L2 - c**2*r2/L2)/L1)/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
    dcc_dcL1=(H*om*((1/2)*N1/L1**2 - 1/2*c**2*r1/L1**2)*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/c + ((1/2)*N1/L1**2 - 1/2*c**2*r1/L1**2)*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/np.sqrt(-N1/L1 + c**2*r1/L1) + L2*np.sqrt(N2/L2 - c**2*r2/L2)/L1**2)/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
    dcc_dcN2=-(1/2)/(L1*np.sqrt(N2/L2 - c**2*r2/L2)*(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2))))
    dcc_dcN1=(-1/2*H*om*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/(L1*c) - 1/2*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
    dcc_dr2=(1/2)*c**2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)*(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2))))
    dcc_dr1=((1/2)*H*c*om*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/L1 + (1/2)*c**2*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))

    # convert dm to real parameters
    dcc_dL1,dcc_dQil1 = dcc_dcL1 * sl1, dcc_dcL1 * L1_r * dsdqil1
    dcc_dL2,dcc_dQil2 = dcc_dcL2 * sl2, dcc_dcL2 * L2_r * dsdqil2
    dcc_dN1,dcc_dQin1 = dcc_dcN1 * sn1, dcc_dcN1 * N1_r * dsdqin1
    dcc_dN2,dcc_dQin2 = dcc_dcN2 * sn2, dcc_dcN2 * N2_r * dsdqin2

    return f,dcc_dL1,dcc_dL2,dcc_dN1,dcc_dN2,dcc_dr1,dcc_dr2,dcc_dQil1,dcc_dQil2

In [8]:
rho1,bv1,bh1,Qv1,Qh1 = 2.800000, 3.00000, 3.3000000, 200., 220.
H,rho2,bv2,bh2,Qv2,Qh2 = 35.000000, 3.200000, 5.00000, 5.5000000, 300., 330.

L1 = rho1**2 * bv1; L2 = rho2**2 * bv2 
N1 = rho1**2 * bh1; N2 = rho2**2 * bh2 

c = 5.32604 * (1.0 +0.0100086j)
om = np.pi * 2. * 0.0005

f = love_func(c,rho1,rho2,L1,L2,N1,N2,Qv1,Qv2,Qh1,Qh2,H,om)

dc = 0.01
f1 = love_func(c * (1+dc),rho1,rho2,L1,L2,N1,N2,Qv1,Qv2,Qh1,Qh2,H,om)
f2 = love_func(c * (1-dc),rho1,rho2,L1,L2,N1,N2,Qv1,Qv2,Qh1,Qh2,H,om)
dfdc_fd = (f1[0] - f2[0]) / (c * dc * 2.)

dL = 0.001
f1 = love_func(c,rho1,rho2,L1,L2,N1*(1+dL),N2,Qv1,Qv2,Qh1,Qh2,H,om)
f2 = love_func(c,rho1,rho2,L1,L2,N1*(1-dL),N2,Qv1,Qv2,Qh1,Qh2,H,om)
dfdL1_fd = (f1[0] - f2[0]) / (N1 * dL * 2.)
dcdL1_fd = - dfdL1_fd / dfdc_fd
print(dcdL1_fd,f[3])


dL = 0.01
f1 = love_func(c,rho1,rho2,L1,L2,N1,N2,Qv1,Qv2*(1+dL),Qh1,Qh2,H,om)
f2 = love_func(c,rho1,rho2,L1,L2,N1,N2,Qv1,Qv2*(1-dL),Qh1,Qh2,H,om)
dfdL1_fd = (f1[0] - f2[0]) / (Qv2 * dL * 2.) * (-Qv2**2)
dcdL1_fd = - dfdL1_fd / dfdc_fd

print(dcdL1_fd,f[8])


(2.5912205805647205e-05-0.0009937968580507202j) (2.5903610027644893e-05-0.0009940105097517608j)
(-0.0002054361164139275-0.13932848266206954j) (-0.0002074415118683403-0.1393443458876129j)
